In [1]:
import importlib

import indra_gpt.chat_curate.chat_curate
importlib.reload(indra_gpt.chat_curate.chat_curate)
from indra_gpt.chat_curate.chat_curate import *

import indra_gpt.chat_curate.eval
importlib.reload(indra_gpt.chat_curate.eval)
from indra_gpt.chat_curate.eval import (curation_statement_evidence_pair_comparison_json, 
                                        curation_statement_comparison_json)

import pickle
from pathlib import Path

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


/Users/thomaslim/miniconda3/envs/indra_gpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO: [2025-07-01 15:59:10] httpx - HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"


In [2]:
RESULTS_DIR = Path.cwd().joinpath("output/curated_statements_sample_100")
if not RESULTS_DIR.exists():
    RESULTS_DIR.mkdir(parents=True)


In [ ]:
import random
import pickle
from pathlib import Path
from tqdm import tqdm
from indra.sources import indra_db_rest

# Step 1: Get all curations from the INDRA DB
print("📥 Fetching all curations from INDRA DB...")
curations = indra_db_rest.get_curations()

# Step 2: Extract unique statement hashes from curations
print("🧹 Extracting unique statement hashes...")
unique_hashes = list({c['pa_hash'] for c in curations if 'pa_hash' in c})

# Step 3: Randomly sample up to 100 curated statement hashes
rng = random.Random(42)  # For reproducibility
sample_size = min(100, len(unique_hashes))
sampled_hashes = rng.sample(unique_hashes, sample_size)

# Step 4: Fetch full statements by hash (including all evidences)
print(f"🔍 Fetching {sample_size} curated statements from INDRA DB...")
stmt_proc = indra_db_rest.get_statements_by_hash(sampled_hashes, ev_limit=1000)
curated_statements = stmt_proc.statements

# Step 5: Save to pickle
output_path = RESULTS_DIR / "statements.pkl"

with open(output_path, "wb") as f:
    pickle.dump(curated_statements, f)

print(f"✅ Saved {len(curated_statements)} curated statements to: {output_path}")


INFO: [2025-07-01 15:59:10] indra.sources.indra_db_rest.util - query: https://db.indra.bio/curation/list
INFO: [2025-07-01 15:59:10] indra.sources.indra_db_rest.util - params: {'api_key': '[api-key]'}
INFO: [2025-07-01 15:59:10] indra.sources.indra_db_rest.util - data: None


📥 Fetching all curations from INDRA DB...


INFO: [2025-07-01 15:59:13] indra_db_rest.query_processor - Retrieving statements that have hash -10696017837805828, -1162888745937699, -12050220369461530, -12574145668536275, -12944136297237273, -13147497679311368, -14588987079463894, -14764107808671838, -15863948540513568, -17373651143639655, -18095852377820217, -18667508778466798, -1916462889756496, -20244182315148832, -20439687097155111, -20640874941404826, -20791049881816836, -20985125539701186, -21063384413635512, -2182577807754994, -22009960213009794, -22089542127820184, -23718381701315206, -24740057688017751, -25687087307736153, -2625142319587850, -28384846765651217, -2925848004089621, -29688714989318986, -29986805880581274, -3014753613862379, -30959821747830391, -31860267491149818, -32408403954411470, -33367977155272434, -33486202106927870, -33901162732123095, -4128417463538137, -4590487136733302, -5222353769539775, -6639728410780169, -6752041385298834, -7750304401378705, -7973912508330969, 1189041814265976, 13315061906556808,

🧹 Extracting unique statement hashes...
🔍 Fetching 100 curated statements from INDRA DB...
✅ Saved 87 curated statements to: /Users/thomaslim/gyorilab/indra_gpt/output/curated_statements_sample_100/statements.pkl


In [4]:
statements = pickle.load(
    open(RESULTS_DIR.joinpath("statements.pkl"), "rb")
)
statements = statements[:5]
models = ['gpt-4o-mini']
for model in models:
    for n_evidence_to_curate in [2, 4, 6, 8, 10]:
        llm_curations_binary = chat_curate_stmts(statements,
                                        n_evidence_to_curate=n_evidence_to_curate,
                                        decision_threshold=0.5,
                                        binary_classification=True,
                                        #ignore_tags=['incorrect'],
                                        n_fewshot_examples=3,
                                        get_ex_per_tag=True,
                                        schema_output_mode=True,
                                        model=model)
        llm_curations_multiclass = chat_curate_stmts(statements,
                                        n_evidence_to_curate=n_evidence_to_curate,
                                        decision_threshold=0.5,
                                        binary_classification=False,
                                        ignore_tags=['incorrect'],
                                        n_fewshot_examples=3,
                                        get_ex_per_tag=True,
                                        schema_output_mode=True,
                                        model=model)

        if not RESULTS_DIR.joinpath(model).joinpath(f"num_evidence_{n_evidence_to_curate}").exists():
            RESULTS_DIR.joinpath(model).joinpath(f"num_evidence_{n_evidence_to_curate}").mkdir(parents=True)
        with open(RESULTS_DIR / model / f"num_evidence_{n_evidence_to_curate}" / "chat_curation_binary_classification.json", "w") as f:
            json.dump(llm_curations_binary, f, indent=2)
        with open(RESULTS_DIR / model / f"num_evidence_{n_evidence_to_curate}" / "chat_curation_multiclass_classification.json", "w") as f:
            json.dump(llm_curations_multiclass, f, indent=2)


Generating prompts...:  20%|██        | 1/5 [00:14<00:57, 14.49s/it]INFO: [2025-07-01 16:00:13] indra_gpt.chat_curate.chat_curate - Skipping evidence with no text for statement -2182577807754994: Evidence(source_api='biogrid',
         pmid='11313979',
         source_id='327840',
         annotations={
                      "biogrid_int_id": "327840",
                      "entrez_a": "5111",
                      "entrez_b": "1026",
                      "biogrid_a": "111142",
                      "biogrid_b": "107460",
                      "syst_name_a": null,
                      "syst_name_b": null,
                      "hgnc_a": "PCNA",
                      "hgnc_b": "CDKN1A",
                      "syn_a": "ATLD2",
                      "syn_b": "CAP20|CDKN1|CIP1|MDA-6|P21|SDI1|WAF1|p21CIP1",
                      "exp_system": "Two-hybrid",
                      "exp_system_type": "physical",
                      "author": "Yu P (2001)",
                      "pmid": "113

In [7]:
for model in models:
    for n_evidence_to_curate in [2, 4, 6, 8, 10]:
        with open(RESULTS_DIR / model / f"num_evidence_{n_evidence_to_curate}" / "chat_curation_binary_classification.json", "r") as f:
            chat_curation_binary_classification = json.load(f)
        with open(RESULTS_DIR / model / f"num_evidence_{n_evidence_to_curate}" / "chat_curation_multiclass_classification.json", "r") as f:
            chat_curation_multiclass_classification = json.load(f)

        eval_stmt_binary = curation_statement_comparison_json(chat_curation_binary_classification)
        with open(RESULTS_DIR / model / f"num_evidence_{n_evidence_to_curate}" / "eval_stmt_chat_curation_binary_classification.json", "w") as f:
            json.dump(eval_stmt_binary, f, indent=2)

        eval_stmt_multi = curation_statement_comparison_json(chat_curation_multiclass_classification)
        with open(RESULTS_DIR / model / f"num_evidence_{n_evidence_to_curate}" / "eval_stmt_chat_curation_multiclass_classification.json", "w") as f:
            json.dump(eval_stmt_multi, f, indent=2)
        
        print(f"Model: {model}, Num Evidence: {n_evidence_to_curate}")
        for eval_strat in ['binary_classification', 'multiclass_classification']:
            with open(RESULTS_DIR / model / f"num_evidence_{n_evidence_to_curate}" / f"eval_stmt_chat_curation_{eval_strat}.json", "r") as f:
                res = json.load(f)
            preds = [res_item['llm_overall_prediction'] for res_item in res]
            labels = [res_item['indra_curation'] for res_item in res]
            acc = accuracy_score(labels, preds)
            print(f"{eval_strat} - Accuracy: {acc:.4f}")


Processing LLM curations:   0%|          | 0/5 [00:00<?, ?it/s]INFO: [2025-07-01 16:18:12] indra.sources.indra_db_rest.util - query: https://db.indra.bio/curation/list/-2925848004089621
INFO: [2025-07-01 16:18:12] indra.sources.indra_db_rest.util - params: {'api_key': '[api-key]'}
INFO: [2025-07-01 16:18:12] indra.sources.indra_db_rest.util - data: None
Processing LLM curations:  20%|██        | 1/5 [00:00<00:00,  4.90it/s]INFO: [2025-07-01 16:18:13] indra.sources.indra_db_rest.util - query: https://db.indra.bio/curation/list/-2182577807754994
INFO: [2025-07-01 16:18:13] indra.sources.indra_db_rest.util - params: {'api_key': '[api-key]'}
INFO: [2025-07-01 16:18:13] indra.sources.indra_db_rest.util - data: None
Processing LLM curations:  40%|████      | 2/5 [00:00<00:00,  3.46it/s]INFO: [2025-07-01 16:18:13] indra.sources.indra_db_rest.util - query: https://db.indra.bio/curation/list/28913319916172234
INFO: [2025-07-01 16:18:13] indra.sources.indra_db_rest.util - params: {'api_key': '[a

Model: gpt-4o-mini, Num Evidence: 2
binary_classification - Accuracy: 0.8000
multiclass_classification - Accuracy: 0.8000


Processing LLM curations:   0%|          | 0/5 [00:00<?, ?it/s]INFO: [2025-07-01 16:18:15] indra.sources.indra_db_rest.util - query: https://db.indra.bio/curation/list/-2925848004089621
INFO: [2025-07-01 16:18:15] indra.sources.indra_db_rest.util - params: {'api_key': '[api-key]'}
INFO: [2025-07-01 16:18:15] indra.sources.indra_db_rest.util - data: None
Processing LLM curations:  20%|██        | 1/5 [00:00<00:00,  5.01it/s]INFO: [2025-07-01 16:18:15] indra.sources.indra_db_rest.util - query: https://db.indra.bio/curation/list/-2182577807754994
INFO: [2025-07-01 16:18:15] indra.sources.indra_db_rest.util - params: {'api_key': '[api-key]'}
INFO: [2025-07-01 16:18:15] indra.sources.indra_db_rest.util - data: None
Processing LLM curations:  40%|████      | 2/5 [00:00<00:00,  3.55it/s]INFO: [2025-07-01 16:18:15] indra.sources.indra_db_rest.util - query: https://db.indra.bio/curation/list/28913319916172234
INFO: [2025-07-01 16:18:15] indra.sources.indra_db_rest.util - params: {'api_key': '[a

Model: gpt-4o-mini, Num Evidence: 4
binary_classification - Accuracy: 0.8000
multiclass_classification - Accuracy: 0.8000


Processing LLM curations:   0%|          | 0/5 [00:00<?, ?it/s]INFO: [2025-07-01 16:18:17] indra.sources.indra_db_rest.util - query: https://db.indra.bio/curation/list/-2925848004089621
INFO: [2025-07-01 16:18:17] indra.sources.indra_db_rest.util - params: {'api_key': '[api-key]'}
INFO: [2025-07-01 16:18:17] indra.sources.indra_db_rest.util - data: None
Processing LLM curations:  20%|██        | 1/5 [00:00<00:00,  4.82it/s]INFO: [2025-07-01 16:18:17] indra.sources.indra_db_rest.util - query: https://db.indra.bio/curation/list/-2182577807754994
INFO: [2025-07-01 16:18:17] indra.sources.indra_db_rest.util - params: {'api_key': '[api-key]'}
INFO: [2025-07-01 16:18:17] indra.sources.indra_db_rest.util - data: None
Processing LLM curations:  40%|████      | 2/5 [00:00<00:00,  4.81it/s]INFO: [2025-07-01 16:18:17] indra.sources.indra_db_rest.util - query: https://db.indra.bio/curation/list/28913319916172234
INFO: [2025-07-01 16:18:17] indra.sources.indra_db_rest.util - params: {'api_key': '[a

Model: gpt-4o-mini, Num Evidence: 6
binary_classification - Accuracy: 0.8000
multiclass_classification - Accuracy: 0.8000


Processing LLM curations:   0%|          | 0/5 [00:00<?, ?it/s]INFO: [2025-07-01 16:18:20] indra.sources.indra_db_rest.util - query: https://db.indra.bio/curation/list/-2925848004089621
INFO: [2025-07-01 16:18:20] indra.sources.indra_db_rest.util - params: {'api_key': '[api-key]'}
INFO: [2025-07-01 16:18:20] indra.sources.indra_db_rest.util - data: None
Processing LLM curations:  20%|██        | 1/5 [00:00<00:00,  5.17it/s]INFO: [2025-07-01 16:18:20] indra.sources.indra_db_rest.util - query: https://db.indra.bio/curation/list/-2182577807754994
INFO: [2025-07-01 16:18:20] indra.sources.indra_db_rest.util - params: {'api_key': '[api-key]'}
INFO: [2025-07-01 16:18:20] indra.sources.indra_db_rest.util - data: None
Processing LLM curations:  40%|████      | 2/5 [00:00<00:00,  5.16it/s]INFO: [2025-07-01 16:18:20] indra.sources.indra_db_rest.util - query: https://db.indra.bio/curation/list/28913319916172234
INFO: [2025-07-01 16:18:20] indra.sources.indra_db_rest.util - params: {'api_key': '[a

Model: gpt-4o-mini, Num Evidence: 8
binary_classification - Accuracy: 0.8000
multiclass_classification - Accuracy: 0.8000


Processing LLM curations:   0%|          | 0/5 [00:00<?, ?it/s]INFO: [2025-07-01 16:18:22] indra.sources.indra_db_rest.util - query: https://db.indra.bio/curation/list/-2925848004089621
INFO: [2025-07-01 16:18:22] indra.sources.indra_db_rest.util - params: {'api_key': '[api-key]'}
INFO: [2025-07-01 16:18:22] indra.sources.indra_db_rest.util - data: None
Processing LLM curations:  20%|██        | 1/5 [00:00<00:00,  4.94it/s]INFO: [2025-07-01 16:18:22] indra.sources.indra_db_rest.util - query: https://db.indra.bio/curation/list/-2182577807754994
INFO: [2025-07-01 16:18:22] indra.sources.indra_db_rest.util - params: {'api_key': '[api-key]'}
INFO: [2025-07-01 16:18:22] indra.sources.indra_db_rest.util - data: None
Processing LLM curations:  40%|████      | 2/5 [00:00<00:00,  4.74it/s]INFO: [2025-07-01 16:18:22] indra.sources.indra_db_rest.util - query: https://db.indra.bio/curation/list/28913319916172234
INFO: [2025-07-01 16:18:22] indra.sources.indra_db_rest.util - params: {'api_key': '[a

Model: gpt-4o-mini, Num Evidence: 10
binary_classification - Accuracy: 0.8000
multiclass_classification - Accuracy: 0.8000


In [9]:
from indra.sources.indra_db_rest import get_curations

get_curations(-17373651143639655)


INFO: [2025-07-01 16:25:31] indra.sources.indra_db_rest.util - query: https://db.indra.bio/curation/list/-17373651143639655
INFO: [2025-07-01 16:25:31] indra.sources.indra_db_rest.util - params: {'api_key': '[api-key]'}
INFO: [2025-07-01 16:25:31] indra.sources.indra_db_rest.util - data: None


[{'curator': 'ben.gyori@gmail.com',
  'date': 'Tue, 15 Dec 2020 15:25:32 GMT',
  'ev_json': None,
  'id': 10446,
  'pa_hash': -17373651143639655,
  'pa_json': None,
  'source': 'DB REST API',
  'source_hash': 3720882788640565537,
  'tag': 'entity_boundaries',
  'text': ''},
 {'curator': 'ben.gyori@gmail.com',
  'date': 'Tue, 15 Dec 2020 15:25:35 GMT',
  'ev_json': None,
  'id': 10447,
  'pa_hash': -17373651143639655,
  'pa_json': None,
  'source': 'DB REST API',
  'source_hash': -1341935765166920808,
  'tag': 'entity_boundaries',
  'text': ''}]

In [ ]:
# from sklearn.metrics import confusion_matrix
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Extract tags
# tags = [c['tag'] for c in curation_comp_json]
# predicted_tags = [c['predicted_tag'] for c in curation_comp_json]

# # Compute accuracy
# accuracy = np.mean([t == p for t, p in zip(tags, predicted_tags)])
# accuracy_bin = np.mean([t == p or (t != 'correct' and p == 'incorrect') for t, p in zip(tags, predicted_tags)])
# print(f"Accuracy: {accuracy:.2%}")
# print(f"Binary Accuracy: {accuracy_bin:.2%}")

# # Define tag label set
# tag_set = list(set(tags).union(set(predicted_tags)))

# # Compute confusion matrix
# cm = confusion_matrix(tags, predicted_tags, labels=tag_set)
# cm_df = pd.DataFrame(cm, index=tag_set, columns=tag_set)

# # Plot
# plt.figure(figsize=(12, 10))
# sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues', cbar=True, linewidths=0.5)
# plt.title("Confusion Matrix (Binary Classification - Model only predicts 'correct' or 'incorrect')")
# plt.xlabel("Predicted Label")
# plt.ylabel("True Label")
# plt.xticks(rotation=45, ha='right')
# plt.yticks(rotation=0)
# plt.tight_layout()
# plt.show()
